# Context-Aware Recommendation Algorithm by PAPERDAA-2

In [96]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from itertools import product as cartesian_product

# Training

## Program Arguments

In [97]:
LMAX = 6      # Maximum transitive path length
K = 10

## Preprocessing

In [98]:
df = pd.read_csv('training_dataset.csv')
df.drop(3520, inplace=True)

df.tail(5)

,userid,itemid,rating,Time,Location,Companion
7033,1082,tt1637706,2,Weekday,Home,Partner
7034,1082,tt1133985,1,Weekday,Home,Partner
7035,1082,tt0413267,1,Weekend,Home,Partner
7036,1082,tt1637706,2,Weekend,Home,Partner
7037,1082,tt0343660,1,Weekend,Home,Partner


In [99]:
encoder = preprocessing.LabelEncoder()

user_item_context_encodings = []
user_item_context_reverse_encodings = []
maximum_rating = df[df.columns[2]].max()

encoded_df = df.copy()
# Encode userid, itemid, and contextual informations for item splitting
for column_index in range(len(df.columns)):
    
    # Column attribute is not rating
    if column_index != 2:
        
        # Fit encoder
        encoder.fit(df[df.columns[column_index]])
        encoded_df[df.columns[column_index]] = encoder.transform(
                df[df.columns[column_index]]
            )
    
    # Column is nor user or rating
    if column_index != 2:
            user_item_context_encodings.append(
                dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
            )
            user_item_context_reverse_encodings.append(
                dict(zip(encoder.transform(encoder.classes_), encoder.classes_))
            )

In [100]:
display(encoded_df)

,userid,itemid,rating,Time,Location,Companion
0,2,38,1,0,0,0
1,2,8,1,0,0,0
2,2,74,1,0,0,0
3,2,5,1,0,0,0
4,2,2,1,0,0,0
...,...,...,...,...,...,...
7033,65,62,2,0,1,2
7034,65,50,1,0,1,2
7035,65,35,1,1,1,2
7036,65,62,2,1,1,2


## Item Splitting

In [101]:
# Cartesian product all items and contexts

users = user_item_context_encodings[0].values()
items = user_item_context_encodings[1].values()
contexts = [
    context_trans.values() for context_trans in user_item_context_encodings[2:]
]

context_T = list(cartesian_product(items, *contexts))

In [102]:
# Generate new user-item matrix for new items
rating_matrix = np.zeros((len(users), len(context_T)), dtype=object)

In [103]:
for row in encoded_df.iterrows():
    data = tuple(row[1])
    user = data[0]
    item = data[1]
    rating = data[2]
    context_item = (item, *data[3:])
    
    index = context_T.index(context_item)
    
    rating_matrix[user][index] = int(rating)/int(maximum_rating)

In [104]:
display(rating_matrix)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0.2, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1.0, 0.8, 1.0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

## Graph Similarity Calculation (User-Based)

In [105]:
rating_matrix_transposed = np.transpose(rating_matrix)

In [106]:
L = 2

WWT = np.matmul(rating_matrix, rating_matrix_transposed)
M = np.matmul(rating_matrix, rating_matrix_transposed)

while L != LMAX:
    M = np.matmul(WWT, M)
    L = L + 2

In [107]:
display(M)

array([[11093.44512, 6925.82848, 2392.134336000001, ...,
        20145.932800000002, 19333.363008000004, 7451.961088000001],
       [6925.828479999999, 15757.451903999994, 3494.885568000001, ...,
        15820.858496000003, 12742.10656, 6772.924544],
       [2392.134336, 3494.8855679999992, 1238.6348800000005, ...,
        4197.259904000001, 7176.677503999998, 2066.4817280000007],
       ...,
       [20145.9328, 15820.858496000004, 4197.2599040000005, ...,
        61203.68774400002, 37503.465727999996, 16856.237760000004],
       [19333.363008, 12742.106560000004, 7176.677504, ..., 37503.465728,
        84074.95270399995, 15402.667583999997],
       [7451.961088000001, 6772.924544, 2066.4817280000007, ...,
        16856.23776, 15402.667584, 8934.016575999998]], dtype=object)

# Prediction

In [108]:
USER = '1082'   # User to check
N = 10

In [109]:
# # Context translation
# translated_context = []

# for cnt_index in range(len(CONTEXT)):
#     # 0 --> User
#     # 1 --> Item
#     # >= 2 --> context
#     map_index = cnt_index + 2
#     translation_table = user_item_context_encodings[map_index]
    
#     translated_context.append(translation_table[CONTEXT[cnt_index]])

# translated_context = tuple(translated_context)
# display(translated_context)

## Ratings Prediction

In [110]:
# Get K most similar users
wanted_user = user_item_context_encodings[0][USER]

scores = M[wanted_user]
K_similar_users = np.argpartition(scores, -(K+1))[-(K+1):]

if wanted_user in K_similar_users:
    K_similar_users = K_similar_users[K_similar_users != wanted_user]
else:
    K_similar_users = K_similar_users[:-1]

### Get list of rated items

In [111]:
inferred_ratings = []
user_rated_items = rating_matrix[wanted_user]

### KNN

In [112]:
for item in range(len(user_rated_items)):
    rating = user_rated_items[item]
    
    # Item has not been rated
    if rating == 0:
        rating_sum = 0
        neighbor_count = 0
        
        for neighbor in K_similar_users:
            neighbor_rating = rating_matrix[neighbor][item]
            
            # If neighbor has rated the item
            if neighbor_rating != 0:
                rating_sum += neighbor_rating
                neighbor_count += 1
            
        inferred_rating = rating_sum / neighbor_count if neighbor_count != 0 else 0
        
        if inferred_rating != 0:
            inferred_ratings.append((item, inferred_rating))

In [113]:
mapped_ratings = []
for i in inferred_ratings:
    
    # Get actual item encoding (from the cartesian product result)
    item_context = context_T[i[0]]
    
    item = user_item_context_reverse_encodings[1][item_context[0]]
    
    new_entry = [item]
    
    for j in range(1, len(item_context)):
        new_entry.append(item_context[j])
    
    new_entry.append(i[1])
    
    mapped_ratings.append(new_entry)

In [114]:
predicted_df = pd.DataFrame(
    mapped_ratings, 
    columns=[
        'Item', 
        *encoded_df.columns[3:], 
        'predicted_rating']
).sort_values(by='predicted_rating', ascending=False)
display(predicted_df)

,Item,Time,Location,Companion,predicted_rating
223,tt0441773,0,1,2,1.0
424,tt3793764,1,1,2,1.0
372,tt1657301,0,1,0,1.0
377,tt1707386,0,0,0,1.0
150,tt0327084,1,1,2,1.0
...,...,...,...,...,...
154,tt0343660,0,1,0,0.2
130,tt0289879,1,0,2,0.2
152,tt0343660,0,0,1,0.2
151,tt0343660,0,0,0,0.2


## Recommendation Generation

In [115]:
# Select N items to be recommended
chosen = []

for data in predicted_df.iterrows():
    data_tup = tuple(data[1])
      
    if len(chosen) <= N:
        chosen.append(data_tup)

for i in chosen:
    print(i)

('tt0441773', 0, 1, 2, 1.0)
('tt3793764', 1, 1, 2, 1.0)
('tt1657301', 0, 1, 0, 1.0)
('tt1707386', 0, 0, 0, 1.0)
('tt0327084', 1, 1, 2, 1.0)
('tt0327084', 0, 1, 0, 1.0)
('tt0319262', 1, 0, 2, 1.0)
('tt0319262', 0, 1, 0, 1.0)
('tt2096673', 0, 0, 0, 1.0)
('tt2096673', 0, 0, 2, 1.0)
('tt0293662', 0, 1, 0, 1.0)


## Rating Prediction Translated Result

In [116]:
mapped_ratings = []
for i in inferred_ratings:
    
    # Get actual item encoding (from the cartesian product result)
    item_context = context_T[i[0]]
    
    item = user_item_context_reverse_encodings[1][item_context[0]]
    
    new_entry = [item]
    
    # Translate remaining context
    for j in range(1, len(item_context)):
        # 1 --> item
        # >= 2 --> context
        translator = user_item_context_reverse_encodings[j+1]
        new_entry.append(translator[item_context[j]])
    
    new_entry.append(i[1])
    
    mapped_ratings.append(new_entry)

In [117]:
res = pd.DataFrame(
    mapped_ratings, 
    columns=[
        'Item', 
        *encoded_df.columns[3:], 
        'predicted_rating']
).sort_values(by='predicted_rating', ascending=False)
display(res)

,Item,Time,Location,Companion,predicted_rating
223,tt0441773,Weekday,Home,Partner,1.0
424,tt3793764,Weekend,Home,Partner,1.0
372,tt1657301,Weekday,Home,Alone,1.0
377,tt1707386,Weekday,Cinema,Alone,1.0
150,tt0327084,Weekend,Home,Partner,1.0
...,...,...,...,...,...
154,tt0343660,Weekday,Home,Alone,0.2
130,tt0289879,Weekend,Cinema,Partner,0.2
152,tt0343660,Weekday,Cinema,Family,0.2
151,tt0343660,Weekday,Cinema,Alone,0.2


In [121]:
res[res['Item'] == 'tt1707386']

,Item,Time,Location,Companion,predicted_rating
377,tt1707386,Weekday,Cinema,Alone,1.000000
382,tt1707386,Weekend,Home,Partner,0.900000
379,tt1707386,Weekend,Cinema,Family,0.866667
378,tt1707386,Weekday,Home,Alone,0.800000
380,tt1707386,Weekend,Cinema,Partner,0.700000
381,tt1707386,Weekend,Home,Family,0.600000
